<a href="https://colab.research.google.com/github/lahirunie-dulsara/EN3150-Assignment-3-CNN/blob/Aazir/appendicities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
pip install ucimlrepo

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import zipfile
# Path to your zip file in Drive
zip_path = '/content/drive/MyDrive/US_Pictures.zip'

# Extract the zip file to a local folder in Colab
extract_dir = '/content/US_Pictures'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Dataset extracted to:", extract_dir)

Mounted at /content/drive
Dataset extracted to: /content/US_Pictures


In [7]:
import os
import re
from PIL import Image

dataset_dir = extract_dir  # path where the dataset was extracted

selected_images = []
selected_filenames = []

# Regular expression: match 'app' or 'appendix' as the only word (after ignoring leading numbers)
pattern = re.compile(r'^(?:[\d\.\s_-]*)(app|appendix)$', re.IGNORECASE)

# Allowed file extensions
allowed_exts = ('.bmp', '.png')

for root, dirs, files in os.walk(dataset_dir):
    for file in files:
        if file.lower().endswith(allowed_exts):
            # Remove file extension
            name_without_ext = os.path.splitext(file)[0]
            # Remove leading/trailing spaces
            cleaned_name = name_without_ext.strip()

            # Match only if the cleaned name fits the pattern
            if pattern.match(cleaned_name):
                img_path = os.path.join(root, file)
                img = Image.open(img_path).convert('L')  # Convert to grayscale
                selected_images.append(img)
                selected_filenames.append(file)

print(f"Total selected images: {len(selected_images)}")

Total selected images: 679


In [8]:
# Path to the folder where you want to save selected images
save_dir = '/content/Selected_Images'

# Create the folder if it doesn't exist
os.makedirs(save_dir, exist_ok=True)

# Loop through selected images and save them
for img, filename in zip(selected_images, selected_filenames):
    save_path = os.path.join(save_dir, filename)
    img.save(save_path)  # PIL will save in the original format automatically

print(f"Saved {len(selected_images)} images to {save_dir}")

Saved 679 images to /content/Selected_Images


In [15]:
from ucimlrepo import fetch_ucirepo
import pandas as pd

# Fetch dataset
regensburg = fetch_ucirepo(id=938)

# Extract features and labels
X = regensburg.data.features
y = regensburg.data.targets

# Combine into one DataFrame
df = pd.concat([X, y], axis=1)
print(df.head())
print(df.columns)


     Age   BMI     Sex  Height  Weight  Length_of_Stay  Alvarado_Score  \
0  12.68  16.9  female   148.0    37.0             3.0             4.0   
1  14.10  31.9    male   147.0    69.5             2.0             5.0   
2  14.14  23.3  female   163.0    62.0             4.0             5.0   
3  16.37  20.6  female   165.0    56.0             3.0             7.0   
4  11.08  16.9  female   163.0    45.0             3.0             5.0   

   Paedriatic_Appendicitis_Score Appendix_on_US  Appendix_Diameter  ...  \
0                            3.0            yes                7.1  ...   
1                            4.0             no                NaN  ...   
2                            3.0             no                NaN  ...   
3                            6.0             no                NaN  ...   
4                            6.0            yes                7.0  ...   

  Bowel_Wall_Thickening Conglomerate_of_Bowel_Loops Ileus Coprostasis  \
0                   NaN        

In [13]:
import os, re

image_dir = '/content/Selected_Images'
image_data = []

for fname in os.listdir(image_dir):
      # extract number before 'app' or 'appendix'
      match = re.search(r'(\d+\.?\d*)\s*(?=.*(app|appendix))', fname, re.IGNORECASE)
      if match:
          img_id = match.group(1)
          image_data.append({'filename': fname, 'ImageNumber': img_id})

image_df = pd.DataFrame(image_data)
print(image_df.head())


        filename ImageNumber
0  742.2 App.bmp       742.2
1  454.2 App.bmp       454.2
2  563.5 App.bmp       563.5
3  506.4 App.bmp       506.4
4  721.3 App.bmp       721.3


In [5]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# Transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load dataset
dataset = datasets.ImageFolder(root=save_dir, transform=transform)

# Split: 70% train, 15% val, 15% test
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_ds, val_ds, test_ds = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)
test_loader = DataLoader(test_ds, batch_size=32)


FileNotFoundError: Couldn't find any class folder in /content/Selected_Images.